In [1]:
import os
import pandas as pd
import sys

install required packages (if needed)

In [ ]:
def install(name):
    import subprocess
    subprocess.call(['pip', 'install', name])
       

all subj folders were downloaded from box and copied into one folder by using this loop:

```
cd [path to downloaded data];
for x in `find 9*/ -name '*.xlsx*' -type f | grep -v template.xlsx`; do echo $x; cp $x /[fullpath to output folder]/; done
```

In [2]:
all_assess_dirs='/Users/daelsaid/scratch/Neuropsych_organized/scratch_testing/all/from_np/'
os.chdir(all_assess_dirs)

#### iterate through folder with all assessments and generate an import xlsx identical to the import formula tab

In [ ]:
for subj_data_file in os.listdir(all_assess_dirs):
    print subj_data_file
    file_prefix=subj_data_file.split('.')[0]
    try:
        subj_df=pd.read_excel(subj_data_file,sheetname='Import_Formula',header=[0],skiprows=None,dtype=str)
        subj_df.fillna(value="",axis=0,inplace=True)
        subj=subj_data_file.split('_')[2]
        visit=subj_data_file.split('.')[0].split('_')[3]
#         subj_df.to_excel(file_prefix+'_import.xlsx',index=False,sheet_name='Import')
    except:
        print "**error for {}".format(file_prefix)
        continue
   

In [12]:
for subj_data_file in os.listdir(all_assess_dirs):
    file_prefix=subj_data_file.split('.')[0]
    if '_import' not in subj_data_file:
        try:
            subj_df=pd.read_excel(subj_data_file,sheetname='Import_Formula',header=[0],skiprows=None,dtype=str)
            subj_df.fillna(value="",axis=0,inplace=True)
            if 'Assessor to complete' or 'Scorer2 to complete' or 'Scorer1 to complete' or 'check' not in subj_df.values:
                print file_prefix
                subj_df.to_excel(file_prefix+'_import_ready.xlsx',index=False,sheet_name='Import')
            else: 
                continue
        except:
            continue


WISC_V_MET_9316_1
MET_WISC_V_9142_1
MET_WIAT_III_9470_1
MET_Geary's_9182_2
MET_WJ_III_9370_1
MET_ATM_9496_2
MET_WASI_II_9142_1
MET_Subtraction_9477_1
MET_WJ_III_9362_1
MET_MARS(SEMA)_9470_1
MET_WASI_II_9492_1
MET_Geary's_9440_2
MET_ATM_9051_1
WJ_III_MET_9302_1
MET_WISC_V_9492_1
WIAT_III_MET_9072_1
MET_ATM_9297_1
MET_ATM_9120_1
MET_Subtraction_9430_1
MET_Subtraction_9302_1
MET_MARS(SEMA)_9302_3
ATL_MET_9032_1
MET_MARS(SEMA)_9390_2
SPT_CBO_9302_1
MET_WJ_III_9145_1
MET_Geary's_9492_1
MET_Subtraction_9496_1
MET_Learning_Curve_9390_1
MET_ATM_9477_2
MET_WJ_III_9002_1
Subtraction_MET_9054_1
MET_ATM_9430_2
MET_Subtraction_9120_2
MET_MARS(SEMA)_9054_3
MET_Subtraction_9297_2
WIAT_III_MET_9347_1
MET_ATM_9347_3
MET_DKEFS_9470_1
MET_Subtraction_9380_1
MET_Geary's_9142_1
MET_ATM_9302_2
WJ_III_MET_9054_1
WIAT_III_MET_9302_1
MET_ATM_9302_3
MET_MARS(SEMA)_9120_1
MET_MARS(SEMA)_9297_1
MET_ATL_9492_1
LearningCurve_MET_9302_1
MET_Subtraction_9297_3
MET_MARS(SEMA)_9051_1
MET_Learning_Curve_9380_1
MET_ATM_9